In [ ]:
!pip3 install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=29f6ee94f8227a697e6a97eebec98fe3705a3a4fa3ab3d325986a221cb374efc
  Stored in directory: /root/.cache/pip/wheels/99/74/83/8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=ec670d0b29203d619df6429bcd585fba3388f753592eee57578a99

In [ ]:
from newspaper import Article, ArticleException
from bs4 import BeautifulSoup
from csv import writer
import pandas as pd
import nltk
import requests
import re
import os

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# **TurnBackHoax Link Scraping**

Diakses pada 22 Januari 2023 pukul 13.30 WIB

In [ ]:
url = "https://turnbackhoax.id/"
page = requests.get(url)

In [ ]:
soup = BeautifulSoup(page.content, 'lxml')

In [ ]:
# write new CSV file
with open('fake_news(link).csv', 'w', encoding='utf8', newline='') as f:
  thewriter = writer(f)
  header = ['Judul', 'URL']
  thewriter.writerow(header)

  links = []

  for article in soup.findAll('h3', {'class' : 'entry-title mh-loop-title'}):
    # links.append(article.a['href'])
    title = getattr(article.find('a'), 'text', None)
    url = article.a['href']

    info = [title, url]
    thewriter.writerow(info)


  for i in range(2,450):
    url = f"https://turnbackhoax.id/page/{i}/"
    page = requests.get(url)

    soup = BeautifulSoup(page.content, 'lxml')

    for article in soup.findAll('h3', {'class' : 'entry-title mh-loop-title'}):
      # links.append(article.a['href'])
      title = getattr(article.find('a'), 'text', None)
      url = article.a['href']

      info = [title, url]
      thewriter.writerow(info)

In [ ]:
df = pd.read_csv("fake_news(link).csv", on_bad_lines='skip')
df

,Judul,URL
0,\n\t\t\t\t\t[DISINFORMASI] “Dana Haji Melayang...,https://turnbackhoax.id/2017/11/30/disinformas...
1,\n\t\t\t\t\t[HOAX] “Akun Twitter Palsu Fahri H...,https://turnbackhoax.id/2017/11/29/hoax-akun-t...
2,\n\t\t\t\t\t[DISINFORMASI] “Garam dan Air Luda...,https://turnbackhoax.id/2017/11/29/disinformas...
3,\n\t\t\t\t\t[EDUKASI] “Sering Minum Es Dapat S...,https://turnbackhoax.id/2017/11/29/edukasi-ser...
4,\n\t\t\t\t\t[ISU] EFEK SAMPING VAKSIN KANKER S...,https://turnbackhoax.id/2017/11/29/isu-efek-sa...
...,...,...
8975,\n\t\t\t\t\t[HOAX] “Penganiayaan Ustadzah di B...,https://turnbackhoax.id/2017/11/24/hoax-pengan...
8976,\n\t\t\t\t\t[HOAX] “31 NOVEMBER GRATIS BENSIN ...,https://turnbackhoax.id/2017/11/24/hoax-31-nov...
8977,\n\t\t\t\t\t[KLARIFIKASI] “Film ‘Naura & Genk ...,https://turnbackhoax.id/2017/11/24/klarifikasi...
8978,\n\t\t\t\t\t[HOAX] “Ratusan Rumah Hanyut di Ma...,https://turnbackhoax.id/2017/11/23/hoax-ratusa...


# **Content Retrieval**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/fake_news_classification_project/Datasets/fake_news.csv"

In [ ]:
links = df['URL'].tolist()

# write new CSV file
with open(path, 'w', encoding='utf8', newline='') as f:
  thewriter = writer(f)
  header = ['Judul', 'Teks', 'Publikasi', 'Ringkasan', 'URL']
  thewriter.writerow(header)

  i = 0
  while i < len(links)/2:
    try:
      # get the article
      article = Article(links[i])

      # do some NLP
      article.download()
      article.parse()
      article.nlp()

      # get the information in article
      title = article.title
      text = article.text
      publish = article.publish_date
      summary = article.summary

      # input the information into csv
      info = [title, text, publish, summary, links[i]]
      thewriter.writerow(info)

    except ArticleException:
      pass

    i += 1

In [ ]:
df = pd.read_csv(path, on_bad_lines='skip')
df

,Judul,Teks,Publikasi,Ringkasan,URL
0,[DISINFORMASI] “Dana Haji Melayang” – TurnBack...,"“Tanggapan dari Nufransa Wira Sakti, Kepala Bi...",2017-11-30 00:00:00,Jadi tidak tepat bila dikatakan bahwa penempat...,https://turnbackhoax.id/2017/11/30/disinformas...
1,[HOAX] “Akun Twitter Palsu Fahri Hamzah” – Tur...,“Tangkapan layar Twitter yang mengatasnamakan ...,2017-11-29 00:00:00,“Tangkapan layar Twitter yang mengatasnamakan ...,https://turnbackhoax.id/2017/11/29/hoax-akun-t...
2,[DISINFORMASI] “Garam dan Air Ludah Untuk Meme...,"“Bukan air ludah + garam, tetapi pecahan keram...",2017-11-29 00:00:00,"“Bukan air ludah + garam, tetapi pecahan keram...",https://turnbackhoax.id/2017/11/29/disinformas...
3,[EDUKASI] “Sering Minum Es Dapat Sebabkan Batu...,"“Dikutip dari vice.com, Andi Khomeini Takdir H...",2017-11-29 00:00:00,"Menurut Andi, minum atau makan es hanya memper...",https://turnbackhoax.id/2017/11/29/edukasi-ser...
4,[ISU] EFEK SAMPING VAKSIN KANKER SERVIKS – Tur...,SUMBER : WHATSAPP\n\nNARASI : INFO DARI KAWAN-...,2017-11-29 00:00:00,SUMBER : WHATSAPPNARASI : INFO DARI KAWAN-KAWA...,https://turnbackhoax.id/2017/11/29/isu-efek-sa...
...,...,...,...,...,...
4485,PKI pada 29 September 2020 di tvOne” – TurnBac...,Hasil Periksa Fakta Natalia Kristian (Anggota ...,2020-09-29 00:00:00,Film Pengkhianatan G30S/PKI akan tayang pada 3...,https://turnbackhoax.id/2020/09/29/salah-poste...
4486,[SALAH] Foto ”Jokowi Mengaku Mirip Ashraf Ingi...,Hasil Periksa Fakta Rizqi Abdul Azis (Anggota ...,2020-09-29 00:00:00,Foto pria sebelah kiri adalah foto lama Jokowi...,https://turnbackhoax.id/2020/09/29/salah-foto-...
4487,[SALAH] “Kartu ATM Nasabah BRI yang berwarna B...,Corporate Secretary BRI Aestika Oryza Gunarto ...,2020-09-29 00:00:00,"Di foto yang ia unggah, terdapat narasi sebaga...",https://turnbackhoax.id/2020/09/29/salah-kartu...
4488,[FALSE] Two Men Dies As A Result Of Police Vio...,Cari untuk:\n\nInformasi Situs ini merupakan a...,2020-09-29 00:00:00,Cari untuk:Informasi Situs ini merupakan arsip...,https://turnbackhoax.id/2020/09/29/false-two-m...
